<a href="https://colab.research.google.com/github/SMenon-14/Terrestrial-Planets-Decision-Forests/blob/main/Telescope_TFDF_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install TensorFlow Decision Forests.
!pip3 install tensorflow_decision_forests --upgrade

# Install Wurlitzer
!pip install wurlitzer

#Import Other Necessary Packages to Reference
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [ ]:
#Checking Decision Forests Version Post-Install
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount ('/content/gdrive',force_remount=True)

In [ ]:
# Load a dataset into a Pandas Dataframe. (Works BC There's less than 1M examples ---- Alternative is using the TensorFlow Dataset to read the files)
dataset_df = pd.read_csv('/content/gdrive/My Drive/Tensorflow Projects/Telescope Project/Train_Set.csv')

# Display the first n examples (n = 4).
dataset_df.head(4)

# This dataset contains a mix of categorical and numerical as well as missing features (Natively supported by TFDF ---- No need for extra processing)

In [ ]:
# Keras expects integer labels but they are currently strings so they must be converted
# Encode the categorical label into an integer. (Necessary for String category labels)

# Name of the label column.
label = "planet_type"

# Create list of possible options in label column
classes = dataset_df[label].unique().tolist()
#Print to Visualize
print(f"Label classes: {classes}")

#Data Frame that contains mapped values of classes indeces for each possible label
dataset_df[label] = dataset_df[label].map(classes.index)

In [ ]:
# Split Data into Training set and Testing Set

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

In [ ]:
# convert the pandas dataframe (pd.Dataframe) into tensorflow datasets (tf.data.Dataset)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [ ]:
# Model Training
# set_cell_height 300

# Specify the model.
model_1 = tfdf.keras.RandomForestModel()

# Optionally, add evaluation metrics.
model_1.compile(
    metrics=["accuracy"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs.
with sys_pipes():
  model_1.fit(x=train_ds)
model_1.summary()

Use /tmp/tmpp7zrt5y0 as temporary training directory
Starting reading the dataset
1/1 [==============================] - ETA: 0s
Dataset read in 0:00:08.509178
Training model
Model trained in 0:00:00.045370
Compiling model


[INFO kernel.cc:1153] Loading model from path
[INFO abstract_model.cc:1063] Engine "RandomForestGeneric" built
[INFO kernel.cc:1001] Use fast generic engine


1/1 [==============================] - 9s 9s/step


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (340):
	ast_flag
	cb_flag
	dec
	decstr
	default_flag
	disc_facility
	disc_instrument
	disc_locale
	disc_pubdate
	disc_refname
	disc_telescope
	disc_year
	discoverymethod
	dkin_flag
	elat
	elon
	etv_flag
	gaia_id
	glat
	glon
	hd_name
	hip_name
	hostname
	htm20
	ima_flag
	micro_flag
	obm_flag
	pl_bmasse
	pl_bmasseerr1
	pl_bmasseerr2
	pl_bmasselim
	pl

In [ ]:
# Model testing
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()
# Viewing Results
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000

loss: 0.0000
accuracy: 1.0000


In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model_1,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1



INFO:tensorflow:Assets written to: /tmp/1/assets


INFO:tensorflow:Assets written to: /tmp/1/assets



Saved model:
total 2252
drwxr-xr-x 2 root root    4096 May 14 17:54 assets
-rw-r--r-- 1 root root  115463 May 14 17:54 keras_metadata.pb
-rw-r--r-- 1 root root 2177590 May 14 17:54 saved_model.pb
drwxr-xr-x 2 root root    4096 May 14 17:54 variables


In [ ]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['ast_flag'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: serving_default_ast_flag:0
    inputs['cb_flag'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: serving_default_cb_flag:0
    inputs['dec'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: serving_default_dec:0
    inputs['decstr'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_decstr:0
    inputs['default_flag'] tens

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  12964      0 --:--:-- --:--:-- --:--:-- 12964
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 66 not upgraded.
Need to get 340 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.8.0 [340 MB]
Fetched 340 MB in 5s (67.8 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.8.0_all.deb ...
Unpacking tensorflow-model-server (2.8.0) ...
Setting up tensorflow-model-server (2.8.0) ...


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=planet_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

2022-05-14 17:55:52.588346: E tensorflow_serving/util/retrier.cc:37] Loading servable: {name: planet_model version: 1} failed: NOT_FOUND: Op type not registered 'SimpleMLCreateModelResource' in binary running on 742d79d25fa6. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.


In [ ]:
# Save the model
model_1.save("/tmp/my_saved_model")

# Plot the first tree
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0, max_depth=3)


model_1.make_inspector().training_logs()

In [ ]:
# Plot Training Curves
import matplotlib.pyplot as plt

logs = model_1.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Log loss (out-of-bag)")

plt.show()

In [ ]:
# Train Keras Model
feature_1 = tfdf.keras.FeatureUsage(name="bill_length_mm")
feature_2 = tfdf.keras.FeatureUsage(name="island")
feature_3 = tfdf.keras.FeatureUsage(name="body_mass_g")
feature_4 = tfdf.keras.FeatureUsage(name="flipper_length_mm")
feature_5 = tfdf.keras.FeatureUsage(name="sex")
feature_6 = tfdf.keras.FeatureUsage(name="year")

all_features = [feature_1, feature_2, feature_3, feature_4, feature_5, feature_6]

# Note: This model is only trained with two features. It will not be as good as
# the one trained on all features.

model_2 = tfdf.keras.GradientBoostedTreesModel(features=all_features, exclude_non_specified_features=True)

model_2.compile(metrics=["accuracy"])
model_2.fit(x=train_ds, validation_data=test_ds)

print(model_2.evaluate(test_ds, return_dict=True))